In [1]:
import pandas as pd

### Read in Data

In [2]:
seeds = pd.read_csv('NCAATourneySeeds.csv')
teams = pd.read_csv('Teams.csv')
regSeason = pd.read_csv('RegularSeasonCompactResults.csv')
regSeasonDetailed = pd.read_csv('RegularSeasonDetailedResults.csv')
tourneyResults = pd.read_csv('NCAATourneyCompactResults.csv')
confTourney = pd.read_csv('ConferenceTourneyGames.csv')

### Narrow down data set to one season

In [3]:
LastSeason = regSeasonDetailed[regSeasonDetailed.Season == 2019]

In [4]:
LastSeason.reset_index(inplace = True)

### Set up ratings dictionary
#### Ratings dictionary will be used keep track of points given to each team to receive a final rating at the end of the program to help decide which teams to pick

In [5]:
ratings = dict()

for i in range(len(LastSeason)):
    if (LastSeason.WTeamID[i] not in ratings):
        ratings[LastSeason.WTeamID[i]] = 0

### Calculate assist - turnover ratio

In [6]:
teamAst = dict()
for i in range(len(LastSeason)):
    if (LastSeason.WTeamID[i] in teamAst):
        teamAst[LastSeason.WTeamID[i]] += LastSeason.WAst[i]
    else:
        teamAst[LastSeason.WTeamID[i]] = LastSeason.WAst[i]
    if (LastSeason.LTeamID[i] in teamAst):
        teamAst[LastSeason.LTeamID[i]] += LastSeason.LAst[i]
    else:
        teamAst[LastSeason.LTeamID[i]] = LastSeason.LAst[i]        

In [7]:
teamTO = dict()
for i in range(len(LastSeason)):
    if (LastSeason.WTeamID[i] in teamTO):
        teamTO[LastSeason.WTeamID[i]] += LastSeason.WTO[i]
    else:
        teamTO[LastSeason.WTeamID[i]] = LastSeason.WTO[i]
    if (LastSeason.LTeamID[i] in teamTO):
        teamTO[LastSeason.LTeamID[i]] += LastSeason.LTO[i]
    else:
        teamTO[LastSeason.LTeamID[i]] = LastSeason.LTO[i]  

In [8]:
teamGames = dict()
for i in range(len(LastSeason)):
    if (LastSeason.WTeamID[i] in teamGames):
        teamGames[LastSeason.WTeamID[i]] += 1
    else:
        teamGames[LastSeason.WTeamID[i]] = 1
    if (LastSeason.LTeamID[i] in teamGames):
        teamGames[LastSeason.LTeamID[i]] += 1
    else:
        teamGames[LastSeason.LTeamID[i]] = 1

In [9]:
teamATOratio = dict()
for i in teamGames:
    teamATOratio[i] = ((teamAst[i] / teamGames[i]) / (teamTO[i] / teamGames[i])) 

### Give ratings to different levels of Assist-Turnover Ratio

In [10]:
for i in teamATOratio:
    if (teamATOratio[i] > 1.5):
        ratings[i] += 15
    elif (teamATOratio[i] > 1.4):
        ratings[i] += 13
    elif (teamATOratio[i] > 1.3):
        ratings[i] += 10
    elif (teamATOratio[i] > 1.2):
        ratings[i] += 8
    elif (teamATOratio[i] > 1.1):
        ratings[i] += 6
    elif (teamATOratio[i] > 1.0):
        ratings[i] += 4
    elif (teamATOratio[i] > 0.9):
        ratings[i] += 2
    elif (teamATOratio[i] > 0.8):
        ratings[i] += 1        

### Calculate Offensive Efficency
#### Possessions = FGA - OR + T + (0.4 * FTA)

In [11]:
teamPoss = dict()
for i in range(len(LastSeason)):
    if (LastSeason.WTeamID[i] in teamPoss):
        teamPoss[LastSeason.WTeamID[i]] += (LastSeason.WFGA[i] - LastSeason.WOR[i] + LastSeason.WTO[i] + (0.4 * LastSeason.WFTA[i]))
    else:
        teamPoss[LastSeason.WTeamID[i]] = (LastSeason.WFGA[i] - LastSeason.WOR[i] + LastSeason.WTO[i] + (0.4 * LastSeason.WFTA[i]))
    if (LastSeason.LTeamID[i] in teamPoss):
        teamPoss[LastSeason.LTeamID[i]] += (LastSeason.LFGA[i] - LastSeason.LOR[i] + LastSeason.LTO[i] + (0.4 * LastSeason.LFTA[i]))
    else:
        teamPoss[LastSeason.LTeamID[i]] = (LastSeason.LFGA[i] - LastSeason.LOR[i] + LastSeason.LTO[i] + (0.4 * LastSeason.LFTA[i]))  

In [12]:
teamPts = dict()
for i in range(len(LastSeason)):
    if (LastSeason.WTeamID[i] in teamPts):
        teamPts[LastSeason.WTeamID[i]] += LastSeason.WScore[i]
    else:
        teamPts[LastSeason.WTeamID[i]] = LastSeason.WScore[i]
    if (LastSeason.LTeamID[i] in teamPts):
        teamPts[LastSeason.LTeamID[i]] += LastSeason.LScore[i]
    else:
        teamPts[LastSeason.LTeamID[i]] = LastSeason.LScore[i]

In [13]:
teamOffEff = dict()
for i in teamPts:
    teamOffEff[i] = ((teamPts[i] / teamPoss[i]) * 100)

### Determine ratings for offensive efficiency

In [14]:
for i in teamOffEff:
    if (teamOffEff[i] > 120):
        ratings[i] += 20
    elif (teamOffEff[i] > 115):
        ratings[i] += 18
    elif (teamOffEff[i] > 110):
        ratings[i] += 13
    elif (teamOffEff[i] > 105):
        ratings[i] += 10
    elif (teamOffEff[i] > 100):
        ratings[i] += 7
    elif (teamOffEff[i] > 95):
        ratings[i] += 4
    elif (teamOffEff[i] > 90):
        ratings[i] += 2

### Calculate Defensive Efficiency

In [15]:
teamOppPoss = dict()
for i in range(len(LastSeason)):
    if (LastSeason.WTeamID[i] in teamOppPoss):
        teamOppPoss[LastSeason.WTeamID[i]] += (LastSeason.LFGA[i] - LastSeason.LOR[i] + LastSeason.LTO[i] + (0.4 * LastSeason.LFTA[i]))
    else:
        teamOppPoss[LastSeason.WTeamID[i]] = (LastSeason.LFGA[i] - LastSeason.LOR[i] + LastSeason.LTO[i] + (0.4 * LastSeason.LFTA[i]))
    if (LastSeason.LTeamID[i] in teamOppPoss):
        teamOppPoss[LastSeason.LTeamID[i]] += (LastSeason.WFGA[i] - LastSeason.WOR[i] + LastSeason.WTO[i] + (0.4 * LastSeason.WFTA[i]))
    else:
        teamOppPoss[LastSeason.LTeamID[i]] = (LastSeason.WFGA[i] - LastSeason.WOR[i] + LastSeason.WTO[i] + (0.4 * LastSeason.WFTA[i]))  

In [16]:
teamOppPts = dict()
for i in range(len(LastSeason)):
    if (LastSeason.WTeamID[i] in teamOppPts):
        teamOppPts[LastSeason.WTeamID[i]] += LastSeason.LScore[i]
    else:
        teamOppPts[LastSeason.WTeamID[i]] = LastSeason.LScore[i]
    if (LastSeason.LTeamID[i] in teamOppPts):
        teamOppPts[LastSeason.LTeamID[i]] += LastSeason.WScore[i]
    else:
        teamOppPts[LastSeason.LTeamID[i]] = LastSeason.WScore[i]

In [17]:
teamDefEff = dict()
for i in teamOppPts:
    teamDefEff[i] = ((teamOppPts[i] / teamOppPoss[i]) * 100)

### Determine Ratings for each level of defensive efficiency

In [18]:
for i in teamDefEff:
    if (teamDefEff[i] < 95):
        ratings[i] += 20
    elif (teamDefEff[i] < 100):
        ratings[i] += 15
    elif (teamDefEff[i] < 105):
        ratings[i] += 9
    elif (teamDefEff[i] < 110):
        ratings[i] += 5
    elif (teamDefEff[i] < 115):
        ratings[i] += 3
    elif (teamDefEff[i] < 120):
        ratings[i] += 1

### Calculate defensive rebounding percentage

In [19]:
teamDR = dict()
for i in range(len(LastSeason)):
    if (LastSeason.WTeamID[i] in teamDR):
        teamDR[LastSeason.WTeamID[i]] += LastSeason.WDR[i]
    else:
        teamDR[LastSeason.WTeamID[i]] = LastSeason.WDR[i]
    if (LastSeason.LTeamID[i] in teamDR):
        teamDR[LastSeason.LTeamID[i]] += LastSeason.LDR[i]
    else:
        teamDR[LastSeason.LTeamID[i]] = LastSeason.LDR[i]

In [20]:
teamOppDR = dict()
for i in range(len(LastSeason)):
    if (LastSeason.WTeamID[i] in teamOppDR):
        teamOppDR[LastSeason.WTeamID[i]] += LastSeason.LDR[i]
    else:
        teamOppDR[LastSeason.WTeamID[i]] = LastSeason.LDR[i]
    if (LastSeason.LTeamID[i] in teamOppDR):
        teamOppDR[LastSeason.LTeamID[i]] += LastSeason.WDR[i]
    else:
        teamOppDR[LastSeason.LTeamID[i]] = LastSeason.WDR[i]

In [21]:
teamDRpct = dict()
for i in teamOppDR:
    teamDRpct[i] = (teamDR[i] / (teamDR[i] + teamOppDR[i]))

### Calculate Offensive Rebounding Percentage

In [22]:
teamOR = dict()
for i in range(len(LastSeason)):
    if (LastSeason.WTeamID[i] in teamOR):
        teamOR[LastSeason.WTeamID[i]] += LastSeason.WOR[i]
    else:
        teamOR[LastSeason.WTeamID[i]] = LastSeason.WOR[i]
    if (LastSeason.LTeamID[i] in teamOR):
        teamOR[LastSeason.LTeamID[i]] += LastSeason.LOR[i]
    else:
        teamOR[LastSeason.LTeamID[i]] = LastSeason.LOR[i]

In [23]:
teamOppOR = dict()
for i in range(len(LastSeason)):
    if (LastSeason.WTeamID[i] in teamOppOR):
        teamOppOR[LastSeason.WTeamID[i]] += LastSeason.LOR[i]
    else:
        teamOppOR[LastSeason.WTeamID[i]] = LastSeason.LOR[i]
    if (LastSeason.LTeamID[i] in teamOppOR):
        teamOppOR[LastSeason.LTeamID[i]] += LastSeason.WOR[i]
    else:
        teamOppOR[LastSeason.LTeamID[i]] = LastSeason.WOR[i]

In [24]:
teamORpct = dict()
for i in teamOppOR:
    teamORpct[i] = (teamOR[i] / (teamOR[i] + teamOppOR[i]))

### Give ratings for levels of each rebounding efficiency

In [25]:
for i in teamDRpct:
    if (teamDRpct[i] > 0.58):
        ratings[i] += 10
    elif (teamDRpct[i] > 0.56):
        ratings[i] += 8
    elif (teamDRpct[i] > 0.54):
        ratings[i] += 6
    elif (teamDRpct[i] > 0.51):
        ratings[i] += 4
    elif (teamDRpct[i] > 0.48):
        ratings[i] += 2
    elif (teamDRpct[i] > 0.45):
        ratings[i] += 1  

In [26]:
for i in teamORpct:
    if (teamORpct[i] > 0.6):
        ratings[i] += 8
    elif (teamORpct[i] > 0.57):
        ratings[i] += 7
    elif (teamORpct[i] > 0.54):
        ratings[i] += 5
    elif (teamORpct[i] > 0.52):
        ratings[i] += 4
    elif (teamORpct[i] > 0.50):
        ratings[i] += 3
    elif (teamORpct[i] > 0.47):
        ratings[i] += 2  
    elif (teamORpct[i] > 0.44):
        ratings[i] += 1

### Give teams in the best 6 conferences 5 extra points because of the difficulty of schedule

In [27]:
conf = dict()
for i in confTourney.index:
    if (confTourney.WTeamID[i] not in conf):
        conf[confTourney.WTeamID[i]] = confTourney.ConfAbbrev[i]
    else:
        continue
    if (confTourney.LTeamID[i] not in conf):
        conf[confTourney.LTeamID[i]] = confTourney.ConfAbbrev[i]
    else:
        continue   

In [28]:
for i in conf:
    if (conf[i] == 'pac_ten' or conf[i] == 'sec' or conf[i] == 'big_east' or conf[i] == 'acc' or 
        conf[i] == 'big_twelve' or conf[i] == 'big_ten'):
        ratings[i] += 5

### Look at ratings and compare to teams

In [29]:
teamRatings = pd.DataFrame.from_dict(ratings, orient = 'index')
teamRatings['Name'] = ''

In [30]:
for i in teamRatings.index:
    for j in range(len(teams)):
        if (teams.TeamID[j] == i):
            teamRatings['Name'][i] = teams.TeamName[j]

c:\users\brand\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [31]:
teamRatings.rename(columns = {0:'Rating'}, inplace = True)

In [32]:
teamRatings.sort_values(by=['Rating'], ascending = False, inplace = True)

### Top 30 Teams

In [33]:
teamRatings[:30]

,Rating,Name
1438,66,Virginia
1211,64,Gonzaga
1277,63,Michigan St
1181,61,Duke
1314,59,North Carolina
1276,57,Michigan
1429,56,Utah St
1246,55,Kentucky
1138,55,Buffalo
1397,54,Tennessee


In [34]:
teamRatings[teamRatings.Name == 'Northeastern']

,Rating,Name
1318,36,Northeastern


In [35]:
teamRatings[teamRatings.Name.str.contains("Colg")]

,Rating,Name
1159,35,Colgate
